## About The Dataset
We will be playing around with the same dataset that we used in the videos.

<strong>The dataset is about the compressive strength of different samples of concrete based on the volumes of the different ingredients that were used to make them. Ingredients include:</strong>

* Cement
* Blast furnace slag
* Fly ash
* Water
* Superplasticizer
* Coarse aggregate
* Fine aggregate

In [1]:
import os
os.environ['TF_ENABLE_ONEDNN_OPTS'] = '0'
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

In [2]:
import pandas as pd
import numpy as np
import keras

import warnings
warnings.simplefilter('ignore', FutureWarning)

In [3]:
filepath='https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0101EN/labs/data/concrete_data.csv'
concrete_data = pd.read_csv(filepath)

concrete_data.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Strength
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28,79.99
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28,61.89
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270,40.27
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365,41.05
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360,44.30


In [4]:
concrete_data.shape

(1030, 9)

In [5]:
concrete_data.describe()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Strength
count,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000
mean,281.167864,73.895825,54.188350,181.567282,6.204660,972.918932,773.580485,45.662136,35.817961
std,104.506364,86.279342,63.997004,21.354219,5.973841,77.753954,80.175980,63.169912,16.705742
min,102.000000,0.000000,0.000000,121.800000,0.000000,801.000000,594.000000,1.000000,2.330000
25%,192.375000,0.000000,0.000000,164.900000,0.000000,932.000000,730.950000,7.000000,23.710000
50%,272.900000,22.000000,0.000000,185.000000,6.400000,968.000000,779.500000,28.000000,34.445000
75%,350.000000,142.950000,118.300000,192.000000,10.200000,1029.400000,824.000000,56.000000,46.135000
max,540.000000,359.400000,200.100000,247.000000,32.200000,1145.000000,992.600000,365.000000,82.600000


In [6]:
concrete_data.isnull().sum()

,0
Cement,0
Blast Furnace Slag,0
Fly Ash,0
Water,0
Superplasticizer,0
Coarse Aggregate,0
Fine Aggregate,0
Age,0
Strength,0


#### Split data into predictors and target


In [7]:
concrete_data_columns = concrete_data.columns

In [8]:
predictors = concrete_data[concrete_data_columns[concrete_data_columns != 'Strength']] # all columns except Strength
target = concrete_data['Strength'] # Strength column

Let's do a quick sanity check of the predictors and the target dataframes.


In [9]:
predictors.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360


In [10]:
target.head()

,Strength
0,79.99
1,61.89
2,40.27
3,41.05
4,44.30


Finally, the last step is to normalize the data by substracting the mean and dividing by the standard deviation.


In [11]:
predictors_norm = (predictors - predictors.mean()) / predictors.std()
predictors_norm.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age
0,2.476712,-0.856472,-0.846733,-0.916319,-0.620147,0.862735,-1.217079,-0.279597
1,2.476712,-0.856472,-0.846733,-0.916319,-0.620147,1.055651,-1.217079,-0.279597
2,0.491187,0.795140,-0.846733,2.174405,-1.038638,-0.526262,-2.239829,3.551340
3,0.491187,0.795140,-0.846733,2.174405,-1.038638,-0.526262,-2.239829,5.055221
4,-0.790075,0.678079,-0.846733,0.488555,-1.038638,0.070492,0.647569,4.976069


In [12]:
n_cols = predictors_norm.shape[1] # number of predictors

##  Import Keras Packages

##### Let's import the rest of the packages from the Keras library that we will need to build our regression model.


In [13]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Input

## Build a Neural Network


In [14]:
# define regression model
def regression_model():
    # create model
    model = Sequential()
    model.add(Input(shape=(n_cols,)))
    model.add(Dense(50, activation='relu'))
    model.add(Dense(50, activation='relu'))
    model.add(Dense(1))

    # compile model
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

## Train and Test the Network


In [15]:
model = regression_model()

In [16]:
model.fit(predictors_norm, target, validation_split=0.3, epochs=100, verbose=2)

Epoch 1/100
23/23 - 2s - 89ms/step - loss: 1652.2781 - val_loss: 1159.1853
Epoch 2/100
23/23 - 0s - 14ms/step - loss: 1529.9452 - val_loss: 1051.2089
Epoch 3/100
23/23 - 0s - 8ms/step - loss: 1324.1233 - val_loss: 868.8542
Epoch 4/100
23/23 - 0s - 9ms/step - loss: 1003.0736 - val_loss: 622.7751
Epoch 5/100
23/23 - 0s - 13ms/step - loss: 627.4617 - val_loss: 388.5403
Epoch 6/100
23/23 - 0s - 13ms/step - loss: 354.6858 - val_loss: 238.1429
Epoch 7/100
23/23 - 0s - 12ms/step - loss: 259.3988 - val_loss: 191.0648
Epoch 8/100
23/23 - 0s - 9ms/step - loss: 233.8159 - val_loss: 176.1878
Epoch 9/100
23/23 - 0s - 13ms/step - loss: 219.4693 - val_loss: 167.1560
Epoch 10/100
23/23 - 0s - 9ms/step - loss: 206.9893 - val_loss: 165.9051
Epoch 11/100
23/23 - 0s - 9ms/step - loss: 198.0560 - val_loss: 158.5553
Epoch 12/100
23/23 - 0s - 13ms/step - loss: 189.7847 - val_loss: 158.1919
Epoch 13/100
23/23 - 0s - 11ms/step - loss: 184.0992 - val_loss: 154.6643
Epoch 14/100
23/23 - 0s - 15ms/step - loss: 17

Now using the same dateset,try to recreate regression model featuring five hidden layers, each with 50 nodes and ReLU activation functions, a single output layer, optimized using the Adam optimizer.

In [17]:
def regression_model():
    input_colm = predictors_norm.shape[1] # Number of input features
    # create model
    model = Sequential()
    model.add(Input(shape=(input_colm,)))  # Set the number of input features
    model.add(Dense(50, activation='relu'))
    model.add(Dense(50, activation='relu'))
    model.add(Dense(50, activation='relu'))
    model.add(Dense(50, activation='relu'))
    model.add(Dense(50, activation='relu'))
    model.add(Dense(1))  # Output layer

    # compile model
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

In [18]:
model = regression_model()
model.fit(predictors_norm, target, validation_split=0.1, epochs=100, verbose=2)

Epoch 1/100
29/29 - 2s - 75ms/step - loss: 1445.8656 - val_loss: 833.6771
Epoch 2/100
29/29 - 0s - 6ms/step - loss: 523.1199 - val_loss: 240.8384
Epoch 3/100
29/29 - 0s - 5ms/step - loss: 226.4393 - val_loss: 214.6624
Epoch 4/100
29/29 - 0s - 5ms/step - loss: 191.6687 - val_loss: 188.5807
Epoch 5/100
29/29 - 0s - 5ms/step - loss: 169.9532 - val_loss: 176.3512
Epoch 6/100
29/29 - 0s - 6ms/step - loss: 153.2171 - val_loss: 151.0593
Epoch 7/100
29/29 - 0s - 6ms/step - loss: 134.6728 - val_loss: 149.4454
Epoch 8/100
29/29 - 0s - 9ms/step - loss: 116.0697 - val_loss: 130.1275
Epoch 9/100
29/29 - 0s - 5ms/step - loss: 97.4099 - val_loss: 110.6109
Epoch 10/100
29/29 - 0s - 5ms/step - loss: 82.9222 - val_loss: 91.0870
Epoch 11/100
29/29 - 0s - 6ms/step - loss: 69.0558 - val_loss: 86.5488
Epoch 12/100
29/29 - 0s - 6ms/step - loss: 62.5634 - val_loss: 86.6254
Epoch 13/100
29/29 - 0s - 9ms/step - loss: 51.8576 - val_loss: 61.1491
Epoch 14/100
29/29 - 0s - 10ms/step - loss: 46.8023 - val_loss: 55.